# Importation des données

##  Librairies (Il y en a surement trop)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import pairwise_distances
import networkx as nx
import itertools
from networkx.drawing.nx_agraph import graphviz_layout
from __future__ import division 
import collections
from networkx.algorithms.richclub import rich_club_coefficient 
import seaborn as sns 
import random 
import math
from operator import itemgetter
from community import community_louvain
from networkx.algorithms import community
from itertools import count
import community

## Données de R

In [6]:
data = pd.read_csv('data.txt', sep="\t")

In [14]:
data.head()

,ID,Qstart,Qend,Sstart,Send
1,1,403127,571184,1,168094
2,2,3774352,3945667,3347618,3518980
3,3,4502860,4651368,4237466,4385991
4,4,1147883,1290190,733701,874035
5,5,2771520,2885776,2355383,2469606


*Id est juste équivalent au numéro de la ligne ce sera plus simple pour nommer les noeuds*

# Création du graph

## Recherche des arc

In [22]:
def find_edges(S,E):
    l = []
    for i in range(len(E)):
        if E.index(E[i]) != S.index(S[i]):
            d = E[i]-S[i]
            print(d)

In [23]:
Qstart = list(data['Qstart'])
Qend = list(data['Qend'])
find_edges(Qstart,Qend )

5426
5430
5430
5351
5351
5359
5359
3512
3380
3304
3304
3304
3304
3303
3304
3303
3303
3303
3303
3303
3303
3303
4075
3719
2043
2034
1962
1967
1960
1957
1987
1957
1963
1956
1956
1963
1956
1956
1958
1961
1955
1955
1955
1955
1955
1959
1961
1955
1955
1955
1960
1955
1955
1955
1955
1970
1955
1955
1957
1954
1954
1954
1954
1954
1954
1957
1954
1957
1954
1954
1957
1954
1954
1954
1954
1954
1954
1960
1954
1954
1954
1954
1954
1963
1957
1954
1962
1964
1954
1954
1954
1954
1954
1956
1959
1956
1954
1954
1954
1954
1954
1954
1960
1954
1954
1954
1954
1956
1956
1956
1954
1954
1954
1954
1954
1954
1954
1954
1958
1954
1954
1954
1954
1954
1957
1954
1954
1954
1954
1954
1954
1956
1956
1956
1956
1956
1956
1957
1954
1954
1954
1954
1954
1954
1954
1962
1957
1957
1957
1957
1954
1954
1954
1954
1954
1954
1954
1954
1954
1954
1957
1954
1954
1954
1960
1957
1954
1954
1954
1954
1954
1954
1961
1957
1954
1956
1953
1953
1953
1953
1953
1953
1953
1953
1953
1953
1953
1953
1953
1953
1953
1953
1953
1953
1953
1953
1953
1953
1953
1956


720
723
714
714
719
715
715
715
715
716
716
716
715
716
715
715
715
715
716
715
716
727
718
718
718
727
715
715
712
712
715
715
719
719
722
718
715
715
712
712
715
715
715
715
712
718
716
715
716
716
716
715
716
715
715
716
715
718
717
718
720
721
714
714
722
718
718
718
718
715
715
715
715
715
715
715
715
715
715
721
714
743
721
722
721
718
715
715
721
721
721
717
717
718
717
717
714
718
718
718
718
715
718
715
715
715
721
722
718
718
715
715
715
715
715
715
715
715
712
719
715
712
712
712
718
715
715
715
712
718
721
735
715
712
712
712
712
712
712
712
724
721
718
715
720
728
724
725
715
715
714
720
717
721
720
717
718
717
718
718
714
720
725
721
718
718
715
715
712
712
712
712
718
717
718
714
714
714
714
718
718
715
715
715
723
718
718
721
727
715
715
715
716
716
716
715
715
715
720
726
717
717
717
717
718
714
714
714
714
714
714
714
714
714
714
714
715
715
715
718
715
716
715
715
716
715
716
715
716
715
716
715
715
716
712
712
712
712
719
716
716
716
716
716
716
716
716
716
717
719


172
171
168
166
166
166
166
166
166
166
166
166
171
168
174
170
168
168
168
168
168
166
166
166
166
166
166
166
166
166
166
166
166
166
177
166
166
166
166
166
166
166
166
172
169
166
166
166
166
166
166
166
166
166
166
166
166
166
166
166
166
166
166
182
166
166
166
166
166
166
166
166
166
166
166
166
166
166
166
168
172
168
168
167
167
167
167
167
167
165
165
165
165
171
165
165
165
165
165
165
176
167
167
165
165
179
165
165
165
174
167
177
167
167
167
169
167
167
167
168
171
167
167
178
172
172
168
165
165
165
165
165
165
165
165
165
165
165
175
168
168
165
168
165
165
165
165
165
165
165
165
165
165
165
165
167
167
167
167
167
165
165
165
165
165
165
165
165
165
165
165
165
165
168
165
168
168
165
165
165
165
165
165
165
165
165
165
165
165
165
167
167
167
167
167
184
180
166
169
166
166
166
166
166
374
374
170
167
167
166
164
164
164
164
164
164
164
191
167
172
167
164
170
166
166
166
166
166
166
166
166
166
166
177
169
166
166
166
166
166
166
166
166
178
167
167
169
166
166
166


166
166
160
173
165
167
167
174
165
165
169
166
167
167
167
171
167
166
166
185
167
179
168
166
166
166
166
166
160
168
176
167
169
171
170
169
165
171
165
166
166
185
166
169
236
164
169
168
168
168
165
170
171
170
167
167
167
166
169
168
167
169
172
169
174
177
167
167
167
176
165
138
166
166
166
166
166
166
166
166
166
166
168
168
166
167
166
167
168
168
165
165
170
166
166
166
166
166
166
166
166
166
166
166
180
172
168
168
165
165
167
167
168
167
168
168
168
168
165
165
168
169
169
173
167
164
130
130
169
134
164
129
129
132
132
129
129
129
129
129
129
129
129
129
129
129
129
129
129
129
129
129
129
129
129
129
129
129
129
129
166
166
166
132
129
129
129
135
132
129
135
129
129
129
129
129
132
129
129
129
129
129
129
129
129
167
166
165
128
128
128
128
128
164
165
165
165
165
165
169
128
131
166
166
131
164
134
134
164
168
128
128
165
165
165
165
130
130
135
130
179
179
134
133
130
130
130
135
132
130
134
168
130
130
134
165
130
130
168
167
130
130
130
167
129
130
133
132
130
130
